## Model 2

In [186]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [187]:
with open(r"data/df_corpus2.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [188]:
df_corpus2.head()

,content,score
0,"Nature Reviews Nephrology (before 2009, Nature...",20.6
1,Food blogging represents a complex interweavin...,13.6
2,The International Motorcycle Shows are a serie...,13.3
3,"Education in Ancient Greece was vastly ""democr...",11.3
4,Rural economics is the study of rural economie...,12.5


## Data Preprocessing

In [189]:
#! pip install gensim

In [190]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sherzyang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [191]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [192]:
processed_docs = df_corpus2['content'].map(preprocess)

In [193]:
# import pickle
# with open('data/processed_docs_lda.pkl', 'wb') as f:
#     pickle.dump(processed_docs, f)

In [194]:
with open(r"data/processed_docs_lda.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [195]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [179]:
# with open("data/dictionary.pkl", "wb") as f: 
#     pickle.dump(dictionary, f)

In [196]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 adult
1 apply
2 cancer
3 child
4 chronic
5 citation
6 clinical
7 coverage
8 diagnosis
9 dialysis
10 disorder


In [197]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [198]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]


In [199]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.11463756621225898),
 (1, 0.05765586532617539),
 (2, 0.13892712182502004),
 (3, 0.09102312571345948),
 (4, 0.1579255683226295),
 (5, 0.15342950456468332),
 (6, 0.12687624510705903),
 (7, 0.13979034449587607),
 (8, 0.15698604788106313),
 (9, 0.11687015539050881),
 (10, 0.19033509306520321),
 (11, 0.0714941413459945),
 (12, 0.10949456875642996),
 (13, 0.15330801836500396),
 (14, 0.2324912851948214),
 (15, 0.2429152820827657),
 (16, 0.07326354021919622),
 (17, 0.10778082117698098),
 (18, 0.1918594902864103),
 (19, 0.24513212262921247),
 (20, 0.13877712452498228),
 (21, 0.20848419321557235),
 (22, 0.09049542779379403),
 (23, 0.27958124452493754),
 (24, 0.18772634012563447),
 (25, 0.10850653149179024),
 (26, 0.16206489016436754),
 (27, 0.05663309758786264),
 (28, 0.1465052227617137),
 (29, 0.05518954915829813),
 (30, 0.2653244279611047),
 (31, 0.11564191336679187),
 (32, 0.2139591889419809),
 (33, 0.32931380435014446),
 (34, 0.17846492643608908),
 (35, 0.09913014119814909)]


In [200]:
with open (r"data/lda_model_1.pkl", "rb") as input_file:
    lda_model = pickle.load(input_file)

In [18]:
#lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [201]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.007*"century" + 0.007*"literature" + 0.006*"write" + 0.004*"book" + 0.004*"displaystyle" + 0.004*"bear" + 0.003*"ancient" + 0.003*"roman" + 0.003*"form" + 0.003*"museum"
Topic: 1 
Words: 0.006*"self" + 0.006*"music" + 0.006*"culture" + 0.006*"people" + 0.005*"health" + 0.004*"psychology" + 0.004*"group" + 0.003*"study" + 0.003*"social" + 0.003*"form"
Topic: 2 
Words: 0.007*"social" + 0.006*"study" + 0.005*"theory" + 0.004*"science" + 0.004*"human" + 0.004*"philosophy" + 0.004*"self" + 0.004*"research" + 0.003*"society" + 0.003*"people"
Topic: 3 
Words: 0.006*"film" + 0.006*"series" + 0.005*"star" + 0.004*"television" + 0.004*"american" + 0.003*"award" + 0.003*"music" + 0.003*"show" + 0.003*"release" + 0.003*"season"
Topic: 4 
Words: 0.009*"university" + 0.007*"program" + 0.007*"engineer" + 0.005*"school" + 0.005*"technology" + 0.005*"science" + 0.004*"college" + 0.004*"education" + 0.004*"research" + 0.004*"institute"
Topic: 5 
Words: 0.008*"food" + 0.006*"company" +

In [202]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.002*"film" + 0.002*"century" + 0.001*"literature" + 0.001*"language" + 0.001*"roman" + 0.001*"ancient" + 0.001*"bear" + 0.001*"movies" + 0.001*"chinese" + 0.001*"museum"
Topic: 1 Word: 0.003*"cities" + 0.002*"music" + 0.002*"psychology" + 0.001*"county" + 0.001*"list" + 0.001*"city" + 0.001*"road" + 0.001*"fitness" + 0.001*"journal" + 0.001*"miles"
Topic: 2 Word: 0.002*"social" + 0.002*"journal" + 0.002*"science" + 0.002*"psychology" + 0.002*"self" + 0.001*"philosophy" + 0.001*"anthropology" + 0.001*"study" + 0.001*"university" + 0.001*"economics"
Topic: 3 Word: 0.004*"film" + 0.003*"music" + 0.002*"award" + 0.002*"actors" + 0.002*"television" + 0.002*"album" + 0.001*"star" + 0.001*"series" + 0.001*"release" + 0.001*"list"
Topic: 4 Word: 0.003*"cities" + 0.002*"population" + 0.002*"list" + 0.001*"language" + 0.001*"census" + 0.001*"food" + 0.001*"languages" + 0.001*"city" + 0.001*"towns" + 0.001*"municipalities"
Topic: 5 Word: 0.003*"food" + 0.003*"finance" + 0.003*"tr

In [21]:
#processed_docs[4310]

In [203]:
#performance evaluation for BOW
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5162047743797302	 
Topic: 0.007*"social" + 0.006*"study" + 0.005*"theory" + 0.004*"science" + 0.004*"human" + 0.004*"philosophy" + 0.004*"self" + 0.004*"research" + 0.003*"society" + 0.003*"people"

Score: 0.31235620379447937	 
Topic: 0.004*"government" + 0.004*"century" + 0.004*"unite" + 0.003*"people" + 0.003*"world" + 0.003*"city" + 0.003*"economic" + 0.003*"empire" + 0.003*"south" + 0.003*"force"

Score: 0.0734516903758049	 
Topic: 0.009*"university" + 0.007*"program" + 0.007*"engineer" + 0.005*"school" + 0.005*"technology" + 0.005*"science" + 0.004*"college" + 0.004*"education" + 0.004*"research" + 0.004*"institute"

Score: 0.0635322704911232	 
Topic: 0.008*"food" + 0.006*"company" + 0.005*"market" + 0.004*"drink" + 0.004*"finance" + 0.004*"service" + 0.003*"world" + 0.003*"unite" + 0.003*"financial" + 0.003*"bank"

Score: 0.03414133936166763	 
Topic: 0.006*"self" + 0.006*"music" + 0.006*"culture" + 0.006*"people" + 0.005*"health" + 0.004*"psychology" + 0.004*"group" + 0

In [204]:
#performance evaluation for TFIDF
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.9881055951118469	 
Topic: 0.002*"social" + 0.002*"journal" + 0.002*"science" + 0.002*"psychology" + 0.002*"self" + 0.001*"philosophy" + 0.001*"anthropology" + 0.001*"study" + 0.001*"university" + 0.001*"economics"

Score: 0.011388801038265228	 
Topic: 0.003*"food" + 0.003*"finance" + 0.003*"transportation" + 0.003*"company" + 0.002*"drink" + 0.002*"service" + 0.002*"market" + 0.002*"global" + 0.001*"bank" + 0.001*"financial"


In [205]:
unseen_document = 'Glossier is a makeup brand that every girl loves.'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model_1[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8197623491287231	 Topic: 0.006*"film" + 0.006*"series" + 0.005*"star" + 0.004*"television" + 0.004*"american"
Score: 0.0200318843126297	 Topic: 0.008*"food" + 0.006*"company" + 0.005*"market" + 0.004*"drink" + 0.004*"finance"
Score: 0.020026803016662598	 Topic: 0.016*"music" + 0.004*"record" + 0.004*"number" + 0.003*"release" + 0.003*"group"
Score: 0.020026493817567825	 Topic: 0.008*"road" + 0.007*"county" + 0.006*"city" + 0.005*"highway" + 0.004*"north"
Score: 0.02002619206905365	 Topic: 0.019*"language" + 0.016*"film" + 0.007*"languages" + 0.005*"write" + 0.005*"word"
Score: 0.02002609334886074	 Topic: 0.007*"century" + 0.007*"literature" + 0.006*"write" + 0.004*"book" + 0.004*"displaystyle"
Score: 0.02002592943608761	 Topic: 0.006*"self" + 0.006*"music" + 0.006*"culture" + 0.006*"people" + 0.005*"health"
Score: 0.020025119185447693	 Topic: 0.007*"social" + 0.006*"study" + 0.005*"theory" + 0.004*"science" + 0.004*"human"
Score: 0.020024782046675682	 Topic: 0.009*"university"

In [36]:
lda_model

[(0, 0.02002611),
 (1, 0.020025944),
 (2, 0.020025136),
 (3, 0.8197604),
 (4, 0.020024799),
 (5, 0.020033775),
 (6, 0.020024315),
 (7, 0.020026209),
 (8, 0.02002651),
 (9, 0.02002682)]

In [37]:
# with open("data/lda_model_1.pkl", 'wb') as f:
#      pickle.dump(lda_model, f)

In [184]:
with open(r"data/lda_model_1.pkl", "rb") as input_file:
    lda_model_1 = pickle.load(input_file)

In [68]:
def get_aggregate_score(lda_model, bow_corpus, i=0):
    """
    Get the score for topics from wikipedia
    """

    lda_all_docs = []   
    
    for i in range(len(bow_corpus)):
        article = lda_model[bow_corpus[i]]
    
        if len(article) != 10:
            indices = []
            for item in article:
                indices.append(item[0])
            for index in list(range(10)):
                if index not in indices:
                    article.append((index, 0))
        
        lda_all_docs.append([i,article])


    return lda_all_docs

In [69]:
aggregate_scores_mod1 = get_aggregate_score(lda_model, bow_corpus, i=0)

In [70]:
topics_df = pd.DataFrame(aggregate_scores_mod1)

In [71]:
topics_df.head()

,0,1
0,0,"[(1, 0.34407565), (2, 0.6388408), (0, 0), (3, ..."
1,1,"[(2, 0.04269844), (3, 0.14525636), (5, 0.80569..."
2,2,"[(3, 0.23120733), (5, 0.7549764), (0, 0), (1, ..."
3,3,"[(0, 0.21230577), (1, 0.03643345), (2, 0.34926..."
4,4,"[(2, 0.27511144), (4, 0.040314987), (5, 0.0807..."


In [99]:
list_of_dicts = []
for i in range(len(topics_df[1])):
    one_doc = dict(topics_df[1][i])
    list_of_dicts.append(one_doc)

In [2]:
#list_of_dicts

In [102]:
topic_matrix_all_wiki = pd.DataFrame(list_of_dicts)

In [103]:
# with open("data/topic_matrix_all_wiki.pkl", 'wb') as f:
#      pickle.dump(topic_matrix_all_wiki, f)

In [ ]:
with open(r"data/topic_matrix_all_wiki.pkl", 'rb') as input_file:
      topic_matrix_all_wiki = pickle.load(input_file)

In [207]:
best_category = []
for i in range(len(list_of_dicts)):
    one_dict = list_of_dicts[i]
    max_value = max(one_dict.values())
    max_keys = [k for k, v in one_dict.items() if v == max_value]
    best_category.append(max_keys)

In [3]:
#best_category 

In [169]:
#[float(str(i).strip('[]')) for i in best_category]

In [162]:
df_best_category = pd.DataFrame()

In [166]:
df_best_category['category'] = [int(str(i).strip('[]')) for i in best_category]

In [167]:
df_best_category['article_index'] = range(len(category))

In [168]:
df_best_category.head()

,category,article_index
0,2,0
1,5,1
2,5,2
3,4,3
4,6,4


In [4]:
# with open("df_best_category.pkl", 'wb') as f:
#      pickle.dump(df_best_category, f)

## Model 3

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import textstat
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [2]:
with open(r"data/new_corpus.pkl", "rb") as input_file:
    new_corpus = pickle.load(input_file)

In [3]:
with open(r"data/df_best_category.pkl", "rb") as input_file:
    df_best_category = pickle.load(input_file)

In [4]:
with open (r"data/dictionary.pkl", "rb") as input_file: 
    dictionary = pickle.load(input_file)

In [5]:
with open (r"data/lda_model_1.pkl", "rb") as input_file:
    lda_model_1 = pickle.load(input_file)

In [6]:
df_best_category.head()

,category,article_index
0,2,0
1,5,1
2,5,2
3,4,3
4,6,4


## Second Function

In [8]:
import textstat
import operator
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from operator import itemgetter

In [9]:
def load_vectorizer(pickle_file='data/vectorizer.pkl'):
    """Loads the trained TF/IDF vectorizer."""
    with open(pickle_file, 'rb') as f:
        return pickle.load(f)
    
def load_corpus_vectors(pickle_file='data/corpus_vectors.pkl'):
    """Loads the corpus vectors."""
    with open(pickle_file, 'rb') as f:
        return pickle.load(f)

In [10]:
def get_vocab_arr(vec):
    n_features = len(vec.vocabulary_)
    vocab_arr = np.empty(n_features, dtype=object)
    for word, idx in vec.vocabulary_.items():
        vocab_arr[idx] = word
    return vocab_arr

In [11]:
def get_top_k_vector(vector, feature_ranking, k=50):
    """Return the top k vector according to feature_ranking."""
    return vector[:, feature_ranking[:k]]
    

In [18]:
def top_50_text(text):
    vec = load_vectorizer()
    corpus_vectors = load_corpus_vectors().toarray()
    sample_vector = vec.transform([text]).toarray()
    feature_ranking = np.argsort(sample_vector[0])[::-1]
    vocab_arr = get_vocab_arr(vec)
    
    distances = cdist(
    get_top_k_vector(sample_vector, feature_ranking),
    get_top_k_vector(corpus_vectors, feature_ranking),
    )
    
    nearest_article_idxs = np.argsort(distances)
    nearest_articles = new_corpus.loc[nearest_article_idxs[0], :]
    top_50 = nearest_articles[:50]
    
    return top_50.sort_values(['score'])

In [19]:
sample = """Data science is the study of the extraction of knowledge from data. It uses various techniques from many fields, including signal processing, mathematics, probability, machine learning, computer programming, statistics, data engineering, pattern matching, and data visualization, with the goal of extracting useful knowledge from the data. With computer systems able to handle more data, big data is an important aspect of data science.

A person that does data science is called a data scientist. Data scientists solve complicated data problems using mathematics, statistics and computer science, although very good skill in these subjects are not required.[1] However, a data scientist is most likely to be an expert in only one or two of these disciplines, meaning that cross disciplinary teams can be a key component of data science.

Good data scientists are able to apply their skills to achieve many kinds of purposes. Their skills and competencies vary widely.
"""

In [43]:
df = top_50_text(sample)

In [44]:
df.reset_index()

,index,score,content
0,13252,7.7,Oregon is a state located in the Western Unite...
1,1233,8.5,Walls And Mirrors is a computer science textbo...
2,6277,12.0,"In computing, the Global File System 2 or GFS2..."
3,11080,12.4,Globalize is a cross-platform JavaScript libra...
4,2539,12.5,Pure Data (Pd) is a visual programming languag...
5,9969,13.1,Scientific Data is a peer-reviewed open access...
6,12275,14.1,"In computer science and computer programming, ..."
7,610,14.4,Digital anthropology is the anthropological st...
8,5831,14.6,A relational database is a digital database ba...
9,10459,14.7,Symbolic regression is a type of regression an...


In [45]:
def get_level_change(x,text):
    """
    Takes in a value and returns the article with the score closest to that value.
    """
    top_50_df = top_50_text(text)
    top_50_df = top_50_df.reset_index()
    top_50_dict = top_50_df['score'].to_dict()
    abs_values = {}
    for key, value in top_50_dict.items():
        temp = abs(value-x)
        abs_values.update({key:temp})
    article_id = min(abs_values, key=abs_values.get)
    level_change = top_50_df['content'][article_id]
    return level_change
    
    

In [59]:
get_level_change(17,sample)

'Wireless sensor network (WSN) refers to a group of spatially dispersed and dedicated sensors for monitoring and recording the physical conditions of the environment and organizing the collected data at a central location. WSNs measure environmental conditions like temperature, sound, pollution levels, humidity, wind, and so on.\nThese are similar to wireless ad hoc networks in the sense that they rely on wireless connectivity and spontaneous formation of networks so that sensor data can be transported wirelessly. WSNs are spatially distributed autonomous sensors to monitor physical or environmental conditions, such as temperature, sound, pressure, etc. and to cooperatively pass their data through the network to a main location. The more modern networks are bi-directional, also enabling control of sensor activity.  The development of wireless sensor networks was motivated by military applications such as battlefield surveillance; today such networks are used in many industrial and cons

## Model 4

In [4]:
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, cdist

In [5]:
with open(r"data/df_corpus2.pkl", "rb") as input_file:
    df_corpus2 = pickle.load(input_file)

In [48]:
vec = TfidfVectorizer(stop_words='english', max_features=10)

In [49]:
vec = vec.fit(df_corpus2['content'])

In [50]:
#vec.vocabulary_

In [51]:
corpus2_vectors = vec.transform(df_corpus2['content']).toarray()

In [52]:
user_doc = "A watermelon is a type of edible fruit, they are 92% water. About 6% of a watermelon is sugar, which makes it very sweet. There are many different types of watermelon. Some have a green rind on the outside and a red-pink flesh on the inside, with brown seeds. Some can have yellow flesh, and some can be seedless. The green rind on the outside is not usually eaten, though it can be used as a vegetable. It can also be stewed or pickled. Most watermelons are oblong or spherical. In Japan, watermelons are grown in different shapes. Many people like to eat watermelon in the summer because the fruit is cool and refreshing. Watermelons are a great source of vitamin A, vitamin C, vitamin B6 and vitamin B1. They also contain potassium, magnesium, carotenoid antioxidant, and lycopene. Watermelons are fruits that come from a vine-like plant."

In [53]:
user_doc_vector = vec.transform([user_doc]).toarray()

In [54]:
user_doc_vector

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.72840959, 0.        , 0.        , 0.68514193, 0.        ]])

In [55]:
distances = cdist(user_doc_vector,
                  corpus2_vectors,
                  metric='cosine')[0]

In [56]:
distances.shape

(14216,)

In [57]:
ranking = np.argsort(distances)

In [58]:
ranking.shape

(14216,)

In [59]:
top_ten = ranking[:10]
top_ten

array([ 3879,  7216,  8986,  1943, 11400, 10636,  9629,  8609,  5384,
        1488])

In [60]:
distances[top_ten]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [61]:
df_corpus2.loc[ranking, 'content']

3879     With a climate as varied and extreme as India,...
7216     International finance (also referred to as int...
8986     A pakudos is a visual motif used by the Mangya...
1943     Construct (psychology), also hypothetical cons...
11400    The epicanthic fold is the skin fold of the up...
10636    Chuan (Chinese: 串, Dungan: Чўан, pinyin: chuàn...
9629     Global Village Tech Park is a software technol...
8609     Tejuíno is a cold beverage made from fermented...
5384     Gens was used by Lewis H. Morgan (in Ancient S...
1488     "Natural Science" is a song by the Canadian ro...
10581    Argument to moderation (Latin: argumentum ad t...
8306     Burgess may refer to:\n\nPeople\nBurgess (name...
11978    Trust for Nature is a not-for-profit organisat...
7376     Kochi is a city in the Indian state of Kerala....
7953     A self-directed Individual Retirement Account ...
6197     Vets Prevail, is a veteran-created online ment...
13135    Humans of some cultures eat octopus. The arms .

## Second Model 

In [10]:
import textstat
import operator
import wikipedia
import wikipediaapi

def give_harder_level(text):
    """
    Takes in text and returns a harder level read
    """
    ##find all the articles that are easier than this article 
    all_harder_text = []
    input_score = textstat.flesch_kincaid_grade(text)
    for i in range(len(df_corpus2['score'])):
        if df_corpus2['score'][i] > input_score:
            all_harder_text.append(df_corpus2['content'][i])
    
    vec = TfidfVectorizer(stop_words='english', max_features=2000)
    vec = vec.fit(all_harder_text)
    corpus2_vectors = vec.transform(all_harder_text).toarray()
    user_doc = text
    user_doc_vector = vec.transform([user_doc]).toarray() 
    user_doc_vector_short = np.argsort(user_doc_vector)[-25:]
    distances = cdist(user_doc_vector_short,
                  corpus2_vectors[-25:],
                  metric='cosine')[0]
    ranking = np.argsort(distances)
    top = ranking[0]
    best_match = df_corpus2['content'][top]
        
    print(distances[top])
    return (best_match)

In [11]:
sample = "On one level, this is a business dispute that highlights tensions in the music industry (both intractably ancient and very current). When Swift signed her six-album deal that furnished Big Machine with rights to her masters, she was partaking in a classic arrangement for new artists: handing over future control of music in exchange for start-up promotional, recording, and distribution help. As she’s risen to megastardom, she’s chafed at that arrangement the same way that many successful musicians have chafed at not having ownership of their work. Prince, who famously protested Warner Bros. Records in 1993 by writing the word slave on his cheek, struck a deal for his own back catalog at great cost in 2014. Paul McCartney preached for years about the importance of artists owning music—and one person he preached to, Michael Jackson, ended up buying the rights to the Beatles’ catalog from under him. West himself recently filed suit to gain control of his own masters."

In [12]:
give_harder_level(sample)

0.6822204939076502


'Value or values may refer to:\n\nValue (philosophy),\nValue (ethics) it may be described as treating actions themselves as abstract objects, putting value to them\nValues (Western philosophy) expands the notion of value beyond that of ethics, but limited to Western sources\nSocial imaginary is the set of values, institutions, laws, and symbols common to a particular social group\nValue (economics), a measure of the benefit that may be gained from goods or service\nTheory of value (economics), the study of the concept of economic value\nValue (marketing), the difference between a customer\'s evaluation of benefits and costs\nValue investing, an investment paradigm\n\nOther uses\nValue, also known as lightness or tone, a representation of variation in the perception of a color or color space\'s brightness\nValue (computer science), an expression that implies no (further) (mathematical) processing; a "normal form"\nValue (mathematics), a property such as number assigned to or calculated 